# 4 Modeling

The stock market is a focus for investors to maximize their potential profits and consequently, the interest shown from the technical and financial sides in stock market prediction is always on the rise.

However, stock market prediction is a problem known for its challenging nature due to its dependency on diverse factors that affect the market, these factors are unpredictable and cannot be taken into consideration such as political variables, and social media effects such as twitter on the stock market.

Text classification can be applied to many different problems, including gauging the public opinion on social media. Historically, it is studied as news articles classification into a pre-determined set of classes, and based on the summary or the title or even the whole body of the article the machine learning models determine if the news article is about economy, sports, real state, and so on. It can also be used for classifying a company's documents into categories that will be analyzed only by the corresponding department. Finally, it can direct, for example, quantify public opinions by classifying the query of a ticker, allowing to gauge public sentiments and lead to accurate price prediction.

## Importing

In [1]:
import os
import math 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#ignore warning messages to ensure clean outputs
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing stock data from previous notebook "03_Preprocessing_and_Training_Data"
eda_stock_data = pd.read_csv('/Users/user/Documents/Springboard_Data_Science/Capstone_2_Twitter_Sentiment_Analysis/Data/eda_stock_data.csv', encoding='latin-1')
eda_stock_data.reset_index(drop=True, inplace=True)
eda_stock_data.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:d418784aaa6e24beb2fce9db3d2414324e4...
1,size 2498


In [3]:
#Importing tweet data from previous notebook "03_Preprocessing_and_Training_Data"
trading_hours_tweets = pd.read_csv('/Users/user/Documents/Springboard_Data_Science/Capstone_2_Twitter_Sentiment_Analysis/Data/trading_hours_tweets.csv', encoding='latin-1')
trading_hours_tweets.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:18151ee3d5f48ae9b237b3c251baebd2502...
1,size 17892920
